In [130]:
import pandas as pd 
from PIL import Image 
import cv2
import numpy as np
import os
import colorsys
import tqdm

# Get main colors

In [131]:
MAIN_COLORS_HSL = {
    'pink': (320, 340),
    'red': ((340, 360), (0, 15)), 'orange': (15, 45),
    'yellow': (45, 55), 'green': (55, 165),
    'blue': (165, 240),
    'violet': (240, 320)
    }

def find_neighbour_color(color_hsv, bl, wh):
    ans = ''
    if bl <= 15 and wh <= 15:
        return 'dark'
    for i, j in MAIN_COLORS_HSL.items():
        if i == 'red':
            if j[0][0] <= color_hsv <= j[0][1] or j[1][0] <= color_hsv <= j[1][1]:
                ans = 'red'
                break
        elif j[0] <= color_hsv <= j[1]:
            ans = i
            break
    if ans == 'red' and ((bl <= 28 and wh >= 78) or wh >= 90):
        ans = 'pink'
    elif ans == 'orange' and  (bl <= 28 and wh >= 78 or wh >= 90):
        ans = 'peach'
    elif (ans == 'orange' or ans == 'red') and (wh <= 25 or (wh <= 45 and bl <= 45)):
        ans = 'brown'
    return ans

In [ ]:
data = pd.read_csv('data.csv', delimiter=';')
data['bl'] = data['bl'] * 100
data['wh'] = data['wh'] / 255 * 100
data['color'] = data.apply(lambda x: find_neighbour_color(x.color_hsv, x.bl, x.wh), axis=1)
data

In [147]:
data.groupby('color').img_path.count()

color
blue      1247
brown      798
dark       492
green      737
orange     986
peach     1314
pink       797
red        575
violet     412
yellow     188
Name: img_path, dtype: int64

In [153]:
for i in data[data.color == 'peach'].img_path[200:400]:
    im = Image.open(i) 
    im.show()


In [155]:
data.to_csv('data_with_colors.csv', index=False)